In [29]:
#!cp -rn /content/aimetodolog /content/drive/MyDrive/prog/manualy/aimetodolog_v2
#!cp -rn /content/drive/MyDrive/prog/manualy/aimetodolog_v2 /content/aimetodolog

In [2]:
# -*- coding: utf-8 -*-
"""
aimetodolog - Нейро-методолог (модульная версия)

Основной ноутбук проекта, использующий модульную архитектуру.
"""

# @title Установка и импорт библиотек
from platform_utils import install_packages
install_packages(['openai', 'json-repair'])

import sys
import os

# Добавляем проект в путь для импорта
# project_path = '/content/aimetodolog'
from platform_utils import get_project_root
project_path = get_project_root()
if project_path not in sys.path:
    sys.path.insert(0, project_path)

print("✅ Библиотеки установлены, пути настроены")

✅ Библиотеки установлены, пути настроены


In [16]:
# @title Импорт модулей проекта

import config
from utils.helpers import format_text, text_to_list_lines, log_to_file, print_header
from core.session_manager import SessionManager
from core.prompt_factory import PromptFactory
from llm.client import get_llm_response
from llm.output_processor import extract_and_repair_json
from utils.structure_parser import parse_structure
from utils.notebook_builder import build_and_save_notebook

print("✅ Модули проекта загружены")
print(f"📁 Проект: {config.PROJECT_NAME}")
print(f"🤖 Модель по умолчанию: {config.DEFAULT_MODEL}")
print(f"📂 Директория логов: {config.LOG_DIR}")

✅ Модули проекта загружены
📁 Проект: aimetodolog
🤖 Модель по умолчанию: tngtech/deepseek-r1t2-chimera:free
📂 Директория логов: /content/logs


In [17]:
# @title Функция диалога с пользователем (обновленная)

def dialog(questions: str) -> str:
    """
    Функция диалога (вопрос-ответ) и сохранение.

    Args:
        questions: вопросы для диалога, каждый вопрос с новой строки

    Returns:
        str: форматированная история диалога
    """
    questions_list = text_to_list_lines(questions)
    dialog_str = ''

    for i, question in enumerate(questions_list, 1):
        formatted_question = format_text(f"Вопрос {i}: {question}")
        print(formatted_question, '\n')
        answer = input("Ваш ответ: ")
        formatted_answer = format_text(f"Ответ: {answer}")
        print()
        dialog_str += f"{formatted_question}\n\n{formatted_answer}\n\n"

    # Логируем диалог
    log_to_file(dialog_str, "user_dialog")

    return dialog_str

In [27]:
# @title Основной рабочий процесс

def main_workflow():
    """Основной рабочий процесс генерации занятия."""

    print_header("НЕЙРО-МЕТОДОЛОГ (модульная версия)")

    # 1. Инициализация сессии
    session = SessionManager(generation_mode=config.DEFAULT_GENERATION_MODE)
    print(f"🆕 Создана сессия: режим '{session.generation_mode}'")

    # 2. Ввод данных пользователя
    print_header("1. Ввод данных пользователя")

    initial_questions = """
1. Какая у Вас будет основная тема занятия(Общая тема: Физика, тема занятия: Закон Архимеда)?
2. Какой у Вас уровень подготовки? (начинающий, средний, продвинутый, начальный (5-7 класс), средний (8-9 класс), продвинутый (10-11 класс), университетский)?
3. Какоя предполагается продолжительность занятия (15 минут, 45 минут, 1 час, 2 часа)?
4. Какими предварительными знаниями и навыками обладают обучаемые в данной теме (никакими, начальными, работаю в данной сфере, являюсь специалистом)?
5. С какой целью хотите изучить занятие(урок в школе, занятие факультатива, лекция на курсе, для самостоятельного самообразования, для профессиональной подготовки, для совершенствования в профессии)?
6. Укажите дополнительные пожелания к занятию: """

    session.summarized_dialog = dialog(initial_questions)
    print(f"💬 Длина диалога: {len(session.summarized_dialog)} символов")

    # 3. Генерация структуры занятия
    print_header("2. Генерация структуры занятия")

    # Создаем промпт для генерации структуры
    structure_system_prompt = """Ты опытный создатель уроков по теме занятия.
Ты должен проанализировать ответы студента на вопросы и создать структуру занятия.
Структура должна включать теоретическую, практическую часть и домашнее задание.
Выведи структуру в формате:
1. Теоретическая часть
   1.1. [название подраздела]
   1.2. [название подраздела]
2. Практическая часть
   2.1. [название подраздела]
   2.2. [название подраздела]
3. Домашнее задание
   3.1. [название подраздела]

Не добавляй никаких дополнительных пояснений, только структуру."""

    structure_user_prompt = f"""Ответы студента: {session.summarized_dialog}

На основе этих ответов создай структуру занятия по теме занятия.
Создай структуру из 3-4 подразделов в каждом основном разделе."""

    print("🧠 Генерация структуры занятия...")

    structure_messages = [
        {"role": "system", "content": structure_system_prompt},
        {"role": "user", "content": structure_user_prompt}
    ]

    structure_raw, structure_time, _ = get_llm_response(
        messages=structure_messages,
        model=config.DEFAULT_MODEL,
        max_tokens=2000
    )

    session.lesson_structure = structure_raw
    print(f"✅ Структура сгенерирована за {structure_time:.2f} сек.")
    print(f"\n📋 Структура занятия:\n{format_text(structure_raw)}")

    # Логируем структуру
    log_to_file(structure_raw, "lesson_structure")

    # 4. Согласование структуры (опционально)
    print_header("3. Согласование структуры")

    need_changes = input("\nХотите внести изменения в структуру? (y/n): ").strip().lower()

    if need_changes in ['y', 'yes', 'да', 'д']:
        changes = input("Опишите изменения: ")

        # Генерация обновленной структуры
        update_system_prompt = """Ты опытный создатель уроков по теме занятия.
Ты должен обновить структуру занятия с учетом пожеланий пользователя."""

        update_user_prompt = f"""Исходная структура: {session.lesson_structure}
Пожелания студента: {changes}
Обнови структуру с учетом пожеланий. Сохрани тот же формат."""

        update_messages = [
            {"role": "system", "content": update_system_prompt},
            {"role": "user", "content": update_user_prompt}
        ]

        updated_structure, update_time, _ = get_llm_response(
            messages=update_messages,
            model=config.DEFAULT_MODEL,
            max_tokens=2000
        )

        session.lesson_structure = updated_structure
        print(f"✅ Структура обновлена за {update_time:.2f} сек.")
        print(f"\n📋 Обновленная структура:\n{format_text(updated_structure)}")

    # 5. Генерация материалов занятия
    print_header("4. Генерация материалов занятия")

    # Инициализируем фабрику промптов
    factory = PromptFactory(session)

    # В зависимости от режима определяем цели генерации
    if session.generation_mode == 'full':
        print("🎯 РЕЖИМ 'FULL': Генерация всего занятия одним запросом")
        # В режиме 'full' цель - вся структура целиком
        generation_targets = [None]  # None означает "весь урок"
        print("   Будет выполнен ОДИН запрос на весь урок")
    else:
        # В режимах 'sections' или 'subsections' парсим структуру
        generation_targets = parse_structure(session.lesson_structure)
        print(f"🎯 РЕЖИМ '{session.generation_mode.upper()}': Генерация по частям")
        print(f"   Будет сгенерировано {len(generation_targets)} разделов")

    # Очищаем ячейки (начинаем с чистого листа)
    session.clear_cells()

    # Цикл генерации
    for i, target in enumerate(generation_targets, 1):
        if session.generation_mode == 'full':
            print(f"\n🔨 Генерация ВСЕГО занятия (запрос {i}/{len(generation_targets)})")
            section_title_display = "ВЕСЬ УРОК"
        else:
            print(f"\n🔨 Генерация раздела {i}/{len(generation_targets)}: {target}")
            section_title_display = target

        # Получаем промпт (для режима 'full' target=None)
        system_prompt, user_prompt = factory.get_prompt(target)

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        # Настраиваем параметры запроса в зависимости от режима
        if session.generation_mode == 'full':
            # Для полной генерации увеличиваем лимит токенов
            current_max_tokens = 8000
            current_temperature = 0.7
        else:
            current_max_tokens = 4000
            current_temperature = 0.7

        # Отправляем запрос к LLM
        print(f"   Параметры: max_tokens={current_max_tokens}, temperature={current_temperature}")
        raw_output, gen_time, _ = get_llm_response(
            messages=messages,
            model=config.DEFAULT_MODEL,
            temperature=current_temperature,
            max_tokens=current_max_tokens
        )

        print(f"   ⏱️  Время генерации: {gen_time:.2f} сек.")

        # Логируем сырой ответ
        log_prefix = "full_lesson" if session.generation_mode == 'full' else f"section_{i}"
        log_to_file(raw_output, log_prefix)

        # Обрабатываем вывод LLM (извлекаем JSON)
        try:
            json_content = extract_and_repair_json(raw_output)

            if 'cells' in json_content:
                session.add_cells(json_content['cells'])
                cell_count = len(json_content['cells'])
                print(f"   ✅ Добавлено {cell_count} ячеек")

                # Для отладки показываем типы ячеек
                cell_types = {}
                for cell in json_content['cells']:
                    cell_type = cell.get('cell_type', 'unknown')
                    cell_types[cell_type] = cell_types.get(cell_type, 0) + 1
                print(f"   📊 Типы ячеек: {cell_types}")
            else:
                print(f"   ⚠️  В ответе нет ячеек (cells)")

        except Exception as e:
            print(f"   ❌ Ошибка обработки JSON: {e}")

    # После цикла выводим статистику
    print(f"\n📈 ИТОГИ ГЕНЕРАЦИИ:")
    print(f"   Режим: {session.generation_mode}")
    print(f"   Всего ячеек: {len(session.cells)}")
    if session.generation_mode == 'full':
        print(f"   Запросов к LLM: 1 (экономия токенов)")
    else:
        print(f"   Запросов к LLM: {len(generation_targets)}")

    # 6. Сборка финального ноутбука
    print_header("5. Сборка финального ноутбука")

    if len(session.cells) == 0:
        print("❌ Нет ячеек для сборки ноутбука")
        return

    notebook_name = input("Введите имя для итогового ноутбука (без .ipynb): ").strip()
    if not notebook_name:
        notebook_name = "generated_lesson"

    notebook_filename = f"{notebook_name}.ipynb"

    # Собираем ноутбук
    notebook_path = build_and_save_notebook(
        cells=session.cells,
        output_dir=session.output_dir,
        filename=notebook_filename
    )

    if notebook_path and os.path.exists(notebook_path):
        print(f"\n🎉 Ноутбук успешно создан!")
        print(f"📁 Путь: {notebook_path}")
        print(f"📊 Ячеек: {len(session.cells)}")

        # Показываем размер файла
        file_size = os.path.getsize(notebook_path)
        print(f"💾 Размер: {file_size / 1024:.1f} KB")

        # Предложение открыть ноутбук
        print("\nВы можете открыть ноутбук в Colab:")
        print(f"  from google.colab import files")
        print(f"  files.download('{notebook_path}')")
    else:
        print("❌ Не удалось создать ноутбук")

    print_header("РАБОТА ЗАВЕРШЕНА")

In [19]:
# @title Вспомогательные функции

def test_module_integration():
    """Тестирует интеграцию всех модулей."""
    print("🧪 Тест интеграции модулей...")

    try:
        # 1. SessionManager
        session = SessionManager()
        session.summarized_dialog = "Тестовый диалог"
        session.lesson_structure = "1. Введение\n2. Практика"
        print("✅ SessionManager: работает")

        # 2. PromptFactory
        factory = PromptFactory(session)
        system_prompt, user_prompt = factory.get_prompt("1. Введение")
        print(f"✅ PromptFactory: создан промпт ({len(system_prompt)} символов)")

        # 3. LLM client (тестовый запрос)
        test_messages = [
            {"role": "system", "content": "Ты помощник. Отвечай кратко."},
            {"role": "user", "content": "Скажи 'Тест пройден'"}
        ]

        response, time_taken, _ = get_llm_response(test_messages, max_tokens=20)
        print(f"✅ LLM client: ответ получен за {time_taken:.2f} сек.")

        # 4. Output processor
        test_json = '{"cells": [{"cell_type": "markdown", "source": ["# Тест"]}]}'
        processed = extract_and_repair_json(test_json)
        print(f"✅ Output processor: обработан JSON ({len(processed.get('cells', []))} ячеек)")

        # 5. Structure parser
        sections = parse_structure(session.lesson_structure)
        print(f"✅ Structure parser: распознано {len(sections)} разделов")

        print("\n🎉 Все модули интегрированы корректно!")

    except Exception as e:
        print(f"❌ Ошибка интеграции: {e}")
        import traceback
        traceback.print_exc()

def quick_save_to_drive():
    """Быстрое сохранение проекта в Google Drive."""
    try:
        from utils.versioning import mount_google_drive, save_version_to_drive

        print("💾 Быстрое сохранение в Google Drive...")

        drive_mounted = mount_google_drive()
        if not drive_mounted:
            print("❌ Google Drive не подключен")
            return

        saved_path = save_version_to_drive(
            project_path='/content/aimetodolog',
            drive_base_path='/content/drive/MyDrive/prog'
        )

        if saved_path:
            print(f"✅ Проект сохранен в: {saved_path}")
        else:
            print("❌ Не удалось сохранить проект")

    except ImportError:
        print("❌ Модуль versioning не найден")
    except Exception as e:
        print(f"❌ Ошибка: {e}")

def show_project_info():
    """Показывает информацию о проекте."""
    import os  # Импорт должен быть в начале функции
    import config

    print("📊 ИНФОРМАЦИЯ О ПРОЕКТЕ")
    print("=" * 50)
    print(f"Название: {config.PROJECT_NAME}")
    print(f"Версия: модульная архитектура")
    print(f"Модель: {config.DEFAULT_MODEL}")
    print(f"Режим генерации: {config.DEFAULT_GENERATION_MODE}")
    print(f"Директория проекта: {config.PROJECT_DIR}")
    print(f"Директория логов: {config.LOG_DIR}")
    print(f"Директория вывода: {config.OUTPUT_DIR}")
    print("=" * 50)

    # Проверяем API ключ
    api_key = os.environ.get("OPENROUTER_API_KEY")
    if api_key:
        print(f"🔑 API ключ: установлен ({len(api_key)} символов)")
    else:
        print("🔑 API ключ: ❌ НЕ УСТАНОВЛЕН")

    # Показываем файлы проекта
    print("\n📁 Содержимое проекта:")
    import os
    for root, dirs, files in os.walk(config.PROJECT_DIR):
        level = root.replace(config.PROJECT_DIR, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in sorted(files)[:5]:  # Показываем первые 5 файлов
            print(f'{subindent}{file}')
        if len(files) > 5:
            print(f'{subindent}... и ещё {len(files) - 5} файлов')

In [20]:
# @title Меню для тестирования и отладки

def test_menu():
    """Меню для тестирования компонентов."""
    while True:
        print("\n" + "=" * 60)
        print("ТЕСТОВОЕ МЕНЮ ПРОЕКТА aimetodolog")
        print("=" * 60)
        print("1. Тест интеграции модулей")
        print("2. Быстрое сохранение в Google Drive")
        print("3. Показать информацию о проекте")
        print("4. Запустить полный рабочий процесс")
        print("5. Выход")
        print("-" * 60)

        choice = input("Выберите действие (1-5): ").strip()

        if choice == '1':
            test_module_integration()
        elif choice == '2':
            quick_save_to_drive()
        elif choice == '3':
            show_project_info()
        elif choice == '4':
            print("\n🚀 Запуск полного рабочего процесса...")
            main_workflow()
            break
        elif choice == '5':
            print("Выход из тестового меню")
            break
        else:
            print("❌ Неверный выбор")

# Запустить тестовое меню
# test_menu()

In [24]:
# @title Утилиты для управления режимами

def set_generation_mode(mode='full'):
    """
    Быстрое переключение режима генерации.

    Args:
        mode: 'full', 'sections' или 'subsections'
    """
    valid_modes = ['full', 'sections', 'subsections']

    if mode not in valid_modes:
        print(f"❌ Недопустимый режим: {mode}. Допустимо: {valid_modes}")
        return

    import config

    # Обновляем конфигурацию
    config.DEFAULT_GENERATION_MODE = mode
    print(f"✅ Режим генерации изменен на: '{mode}'")

    # Создаем тестовую сессию с новым режимом
    session = SessionManager(generation_mode=mode)
    factory = PromptFactory(session)

    print(f"📊 Характеристики режима '{mode}':")
    if mode == 'full':
        print("   • 1 запрос на весь урок")
        print("   • Экономия токенов (меньше overhead)")
        print("   • Быстрая генерация")
        print("   • Риск: ответ может быть слишком большим")
    elif mode == 'sections':
        print("   • Запрос для каждого раздела 1-го уровня")
        print("   • Более контролируемая генерация")
        print("   • Умеренное использование токенов")
    elif mode == 'subsections':
        print("   • Запрос для каждого подраздела")
        print("   • Максимальная детализация")
        print("   • Высокий расход токенов")
        print("   • Медленная генерация")

    return session, factory

# Пример использованsия:
# set_generation_mode('full')  # Для быстрого тестирования
# set_generation_mode('sections')  # Для баланса
# set_generation_mode('subsections')  # Для детальной проработки

In [22]:
# @title Тест разных режимов генерации

def test_modes():
    """Тестирует все режимы генерации."""

    test_modes_list = ['full', 'sections', 'subsections']

    for mode in test_modes_list:
        print(f"\n{'='*60}")
        print(f"ТЕСТ РЕЖИМА: {mode.upper()}")
        print(f"{'='*60}")

        # Создаем тестовую сессию
        session = SessionManager(generation_mode=mode)
        session.summarized_dialog = "Тест: создание урока по нейросетям"
        session.lesson_structure = "1. Введение\n2. Практика\n3. ДЗ"

        # Создаем фабрику
        factory = PromptFactory(session)

        # Получаем промпты
        try:
            if mode == 'full':
                system_prompt, user_prompt = factory.get_prompt()
                print(f"✅ Промпт для режима 'full':")
                print(f"   System: {system_prompt[:100]}...")
                print(f"   User: {user_prompt}")
            else:
                # Для sections/subsections нужен target
                target = "1. Введение" if mode == 'sections' else "1.1. Основы"
                system_prompt, user_prompt = factory.get_prompt(target)
                print(f"✅ Промпт для режима '{mode}' (target='{target}'):")
                print(f"   System: {system_prompt[:100]}...")
                print(f"   User: {user_prompt}")

            print(f"✓ Режим '{mode}' работает корректно")

        except Exception as e:
            print(f"✗ Ошибка в режиме '{mode}': {e}")

# Запустить тест
# test_modes()

In [23]:
test_modes()


ТЕСТ РЕЖИМА: FULL
✅ Промпт для режима 'full':
   System: Ты — опытный создатель уроков по нейросетям на Python для Google Colab.
Твоя задача — создать качест...
   User: Сгенерируй полный Jupyter Notebook для всего занятия.
✓ Режим 'full' работает корректно

ТЕСТ РЕЖИМА: SECTIONS
✅ Промпт для режима 'sections' (target='1. Введение'):
   System: Ты — опытный создатель уроков по нейросетям на Python для Google Colab.
Твоя задача — создать качест...
   User: Сгенерируй материал для раздела: '1. Введение'
✓ Режим 'sections' работает корректно

ТЕСТ РЕЖИМА: SUBSECTIONS
✅ Промпт для режима 'subsections' (target='1.1. Основы'):
   System: Ты — опытный создатель уроков по нейросетям на Python для Google Colab.
Твоя задача — создать качест...
   User: Сгенерируй детализированный материал для подраздела: '1.1. Основы'
✓ Режим 'subsections' работает корректно


In [25]:
set_generation_mode('full')

✅ Режим генерации изменен на: 'full'
📊 Характеристики режима 'full':
   • 1 запрос на весь урок
   • Экономия токенов (меньше overhead)
   • Быстрая генерация
   • Риск: ответ может быть слишком большим


(<core.session_manager.SessionManager at 0x7c9f328627e0>,
 <core.prompt_factory.PromptFactory at 0x7c9f328622d0>)

In [28]:
# @title Запуск основного рабочего процесса

if __name__ == "__main__":
    try:
        main_workflow()
    except KeyboardInterrupt:
        print("\n\n⚠️  Прервано пользователем")
    except Exception as e:
        print(f"\n❌ Критическая ошибка: {e}")
        import traceback
        traceback.print_exc()


====================== НЕЙРО-МЕТОДОЛОГ (модульная версия) ======================
🆕 Создана сессия: режим 'full'

========================= 1. Ввод данных пользователя ==========================
Вопрос 1: 1. Какая у Вас будет основная тема занятия(Общая тема: Физика, тема занятия: Закон Архимеда)? 

Ваш ответ: Общая тема: Физика, тема занятия: Закон Архимеда

Вопрос 2: 2. Какой у Вас уровень подготовки? (начинающий, средний, продвинутый, начальный (5-7 класс), средний (8-9
класс), продвинутый (10-11 класс), университетский)? 

Ваш ответ: средний

Вопрос 3: 3. Какоя предполагается продолжительность занятия (15 минут, 45 минут, 1 час, 2 часа)? 

Ваш ответ: 15 минут

Вопрос 4: 4. Какими предварительными знаниями и навыками обладают обучаемые в данной теме (никакими, начальными, работаю
в данной сфере, являюсь специалистом)? 

Ваш ответ: начальными

Вопрос 5: 5. С какой целью хотите изучить занятие(урок в школе, занятие факультатива, лекция на курсе, для
самостоятельного самообразования, д

In [10]:
test_menu()


ТЕСТОВОЕ МЕНЮ ПРОЕКТА aimetodolog
1. Тест интеграции модулей
2. Быстрое сохранение в Google Drive
3. Показать информацию о проекте
4. Запустить полный рабочий процесс
5. Выход
------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [9]:
# @title Быстрый тест интеграции

print("🧪 Быстрый тест интеграции модулей...")

try:
    # Импорт
    from core.session_manager import SessionManager
    from core.prompt_factory import PromptFactory

    # Тест SessionManager
    session = SessionManager()
    print(f"✅ SessionManager создан (режим: {session.generation_mode})")

    # Тест PromptFactory
    factory = PromptFactory(session)
    print(f"✅ PromptFactory создан")

    # Получение промпта
    system_prompt, user_prompt = factory.get_prompt("1.1. Введение")
    print(f"✅ Промпт получен (системный: {len(system_prompt)} chars)")

    print("\n🎉 Базовая интеграция работает!")

except Exception as e:
    print(f"❌ Ошибка: {e}")
    import traceback
    traceback.print_exc()

🧪 Быстрый тест интеграции модулей...
✅ SessionManager создан (режим: full)
✅ PromptFactory создан
✅ Промпт получен (системный: 531 chars)

🎉 Базовая интеграция работает!
